In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

PATH = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered"

In [2]:
##setting the dir of the datasets; 
train_dir = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered/train"
validation_dir = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered/validation"

train_cats_dir = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered/train/cats"  # directory with our training cat pictures
train_dogs_dir = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered/train/dogs"  # directory with our training dog pictures
validation_cats_dir = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered/validation/cats"  # directory with our validation cat pictures
validation_dogs_dir = "D:/study/2020 Spring/597 Deep Learning/dataset/cats_and_dogs_filtered/validation/dogs"  # directory with our validation dog pictures

In [3]:
#take a peek at the data; 
print(len(os.listdir(train_cats_dir)))
#load images using ImageDataGenerator; 
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
batch_size = 1000
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')
validation_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

1000
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [18]:
import tensorflow.keras as keras
from keras import models
from keras import layers
sample_training_images, sample_training_labels = next(train_data_gen)
sample_validation_images, sample_validation_labels = next(validation_data_gen)
sample_training_images = sample_training_images.reshape((1000, 150 * 150 *3))
sample_training_images = sample_training_images.astype('float32') / 255
sample_validation_images = sample_validation_images.reshape((1000, 150 * 150 *3))
sample_validation_images = sample_validation_images.astype('float32') / 255


In [5]:
## Logistic regression

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

clf = LogisticRegression()
clf.fit(sample_training_images,sample_training_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
test_predict = clf.predict(sample_validation_images)
print("Accuracy:",metrics.accuracy_score(sample_validation_labels, test_predict))

Accuracy: 0.583


In [7]:
## Optimal separating hyperplane using SVM
from sklearn import svm

clf = svm.SVC(kernel='linear')
clf.fit(sample_training_images,sample_training_labels)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [8]:
test_predict = clf.predict(sample_validation_images)
print("Accuracy:",metrics.accuracy_score(sample_validation_labels, test_predict))

Accuracy: 0.573


In [19]:
## Logistic regression (binary) using NN

netLogiB = models.Sequential() # Sequential feed forward NN
netLogiB.add(layers.Dense(1, activation='sigmoid', input_shape=(150*150*3,)))
netLogiB.compile(optimizer='rmsprop',loss='mean_squared_error',metrics=['accuracy'])
netLogiB.fit(sample_training_images, sample_training_labels, epochs=5, batch_size=128)

Epoch 1/5
1000/1000 [==============================] - 0s 437us/step - loss: 0.2494 - acc: 0.5210
Epoch 2/5
1000/1000 [==============================] - 0s 303us/step - loss: 0.2484 - acc: 0.5500
Epoch 3/5
1000/1000 [==============================] - 0s 301us/step - loss: 0.2473 - acc: 0.5530
Epoch 4/5
1000/1000 [==============================] - 0s 299us/step - loss: 0.2459 - acc: 0.5610
Epoch 5/5
1000/1000 [==============================] - 0s 303us/step - loss: 0.2461 - acc: 0.5500


In [20]:
test_loss, test_acc = netLogiB.evaluate(sample_validation_images, sample_validation_labels)
print(test_acc)

1000/1000 [==============================] - 0s 362us/step
0.582


In [21]:
from keras.utils import to_categorical
sample_training_labels = to_categorical(sample_training_labels)
sample_validation_labels = to_categorical(sample_validation_labels)

In [12]:
## Logistic regression (multinomial) using NN

netLogiM = models.Sequential() # Sequential feed forward NN
netLogiM.add(layers.Dense(2, activation='sigmoid', input_shape=(150*150*3,)))
netLogiM.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
netLogiM.fit(sample_training_images, sample_training_labels, epochs=5, batch_size=128)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/5
1000/1000 [==============================] - 1s 516us/step - loss: 0.6956 - acc: 0.5010
Epoch 2/5
1000/1000 [==============================] - 0s 400us/step - loss: 0.6937 - acc: 0.5540
Epoch 3/5
1000/1000 [==============================] - 0s 396us/step - loss: 0.6903 - acc: 0.5500
Epoch 4/5
1000/1000 [==============================] - 0s 391us/step - loss: 0.6880 - acc: 0.5420
Epoch 5/5
1000/1000 [==============================] - 0s 384us/step - loss: 0.6876 - acc: 0.5210


In [13]:
test_loss, test_acc = netLogiM.evaluate(sample_validation_images, sample_validation_labels)
print(test_acc)

1000/1000 [==============================] - 0s 312us/step
0.524


In [14]:
## Linear regression using NN

netLin = models.Sequential() # Sequential feed forward NN
netLin.add(layers.Dense(2, activation='linear', input_shape=(150*150*3,)))
netLin.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
netLin.fit(sample_training_images, sample_training_labels, epochs=5, batch_size=128)

Epoch 1/5
1000/1000 [==============================] - 1s 536us/step - loss: 8.1250 - acc: 0.4970
Epoch 2/5
1000/1000 [==============================] - 0s 400us/step - loss: 8.2202 - acc: 0.4970
Epoch 3/5
1000/1000 [==============================] - 0s 396us/step - loss: 8.2202 - acc: 0.4970
Epoch 4/5
1000/1000 [==============================] - 0s 396us/step - loss: 8.2202 - acc: 0.4970
Epoch 5/5
1000/1000 [==============================] - 0s 404us/step - loss: 8.2202 - acc: 0.4970


In [15]:
test_loss, test_acc = netLin.evaluate(sample_validation_images, sample_validation_labels)
print(test_acc)

1000/1000 [==============================] - 0s 321us/step
0.509


In [24]:
## Optimal separating hyperplane using NN

netSplane = models.Sequential()
netSplane.add(layers.Dense(2, activation='linear', kernel_regularizer=keras.regularizers.l2()))

# Train the model
netSplane.compile(optimizer='adam', loss="hinge",metrics=['accuracy'])
netSplane.fit(sample_training_images, sample_training_labels, epochs=5, batch_size=128)


Epoch 1/5
1000/1000 [==============================] - 1s 656us/step - loss: 0.8609 - acc: 0.5230
Epoch 2/5
1000/1000 [==============================] - 0s 443us/step - loss: 0.6838 - acc: 0.4970
Epoch 3/5
1000/1000 [==============================] - 0s 441us/step - loss: 0.6672 - acc: 0.5340
Epoch 4/5
1000/1000 [==============================] - 0s 422us/step - loss: 0.6394 - acc: 0.5030
Epoch 5/5
1000/1000 [==============================] - 0s 414us/step - loss: 0.6391 - acc: 0.5050


In [25]:
test_loss, test_acc = netSplane.evaluate(sample_validation_images, sample_validation_labels)
print(test_acc)

1000/1000 [==============================] - 0s 379us/step
0.509
